
**1. Install packages to cloud**

In [ ]:
%pip install azure.storage.blob

     |████████████████████████████████| 346 kB 4.4 MB/s 
     |████████████████████████████████| 178 kB 43.4 MB/s 
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 4.0 MB 47.6 MB/s 
     |████████████████████████████████| 41 kB 576 kB/s 


In [ ]:
%pip install azure.ai.formrecognizer

     |████████████████████████████████| 149 kB 4.2 MB/s 



**2. Connect to Azure Storage Container** 

In [ ]:
from azure.storage.blob import ContainerClient

container_url = "https://xxxxxxxxx/testinvoice"
container = ContainerClient.from_container_url(container_url)

for blob in container.list_blobs():
 
  blob_url = container_url + "/" + blob.name
  print(blob_url)

**3. Enable Congitive Services**

In [ ]:
import requests
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

endpoint = "https://xxxxxxxx/"
credential = AzureKeyCredential("xxxxxxxxxxxxxxxx")
form_recognizer_client = FormRecognizerClient(endpoint, credential)
print(form_recognizer_client)

# endpoint = dbutils.secrets.get(scope="formrec",key="https://ocr-invoice-multiline.cognitiveservices.azure.com/")
# key = dbutils.secrets.get(scope="formrec",key="6ba97bb2df58483aa6a17fa2db9914ab")

# form_recognizer_client = FormRecognizerClient(endpoint=endpoint, credential=AzureKeyCredential(key))

**4. Create dataframe of invoice data.**

In [ ]:
import pandas as pd

field_list = ["InvoiceId", "VendorName", "VendorAddress", "CustomerName", "CustomerAddress", "CustomerAddressRecipient", "InvoiceDate", "InvoiceTotal", "DueDate"]
df = pd.DataFrame(columns=field_list)

for blob in container.list_blobs():
  blob_url = container_url + "/" + blob.name

  poller = form_recognizer_client.begin_recognize_invoices_from_url(blob_url)
  # poller = form_recognizer_client.begin_recognize_invoices_from_url(invoice_url=blob_url)
  invoices = poller.result()
  print("Scanning " + blob.name + "...")
  
  for idx, invoice in enumerate(invoices):
      single_df = pd.DataFrame(columns=field_list)

      for field in field_list:
        entry = invoice.fields.get(field)
        
        if entry:
          single_df[field] = [entry.value]
          
      single_df['FileName'] = blob.name
      df = df.append(single_df)

df = df.reset_index(drop=True)
df

Scanning HARBHOLETEMPOSERVICE.png...
Scanning LOGIX.pdf...


,InvoiceId,VendorName,VendorAddress,CustomerName,CustomerAddress,CustomerAddressRecipient,InvoiceDate,InvoiceTotal,DueDate,FileName
0,9167364045,HAR BHOLE TEMPO SERVICE,"Shop No. 3, Sai Niketan, Bhattwadi, Kisan Naga...",NaN,NaN,NaN,NaN,27400.0,NaN,HARBHOLETEMPOSERVICE.png
1,ILPL212200774 ILPL212200774,Logix Infosecurity Pvt.Ltd.,"Unit no.127, bldg. No. 2 (A-3), Sector - 1, Mi...",MULTILINE BUSINESS SOLUTION PVT LTD,"1st Floor, Parekh Print, Itilian Compound, IIT...",MULTILINE BUSINESS SOLUTION PVT LTD,None,0.0,NaN,LOGIX.pdf


In [ ]:
df.to_csv('invoice_data.csv')

In [ ]:
# account_name = "formrecognizerdemo070621"
# account_key = "fs.azure.account.key." + account_name + ".blob.core.windows.net"

# try: 
#   dbutils.fs.mount(
#     source = "wasbs://pdf-recognized@formrecognizerdemo070621.blob.core.windows.net",
#     mount_point = "/mnt/pdf-recognized",
#     extra_configs = {account_key: dbutils.secrets.get(scope = "formrec", key = "formreckey")} )

# except:
#   print('Directory already mounted or error')

# df.to_csv(r"/dbfs/mnt/pdf-recognized/output.csv", index=False)
